## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Punta Arenas,-53.1500,-70.9167,44.11,52,35,25.43,CL,scattered clouds
1,1,Constantine,36.3650,6.6147,58.59,67,63,3.62,DZ,broken clouds
2,2,Broken Hill,-31.9500,141.4333,52.59,55,5,4.81,AU,clear sky
3,3,Los Llanos De Aridane,28.6585,-17.9182,72.12,80,20,6.91,ES,volcanic ash
4,4,Uvira,-3.4067,29.1458,75.63,80,35,1.59,CD,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
4,4,Uvira,-3.4067,29.1458,75.63,80,35,1.59,CD,scattered clouds
8,8,Rikitea,-23.1203,-134.9692,75.00,79,26,9.62,PF,scattered clouds
12,12,Mount Vernon,40.9126,-73.8371,80.98,74,36,4.00,US,scattered clouds
15,15,Albany,42.6001,-73.9662,75.09,71,99,1.99,US,overcast clouds
16,16,Sao Filipe,14.8961,-24.4956,79.34,79,100,13.60,CV,overcast clouds
20,20,Namatanai,-3.6667,152.4333,81.23,77,32,9.95,PG,scattered clouds
25,25,Mecca,21.4267,39.8261,85.91,46,0,4.56,SA,clear sky
27,27,Ponta Do Sol,32.6667,-17.1000,75.70,58,83,4.76,PT,broken clouds
30,30,Ambovombe,-25.1667,46.0833,77.52,60,3,6.15,MG,clear sky
31,31,Ugoofaaru,5.6667,73.0000,81.82,77,100,24.07,MV,overcast clouds


In [34]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Current Description    0
dtype: int64

In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [23]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Uvira,CD,75.63,scattered clouds,-3.4067,29.1458,
8,Rikitea,PF,75.00,scattered clouds,-23.1203,-134.9692,
12,Mount Vernon,US,80.98,scattered clouds,40.9126,-73.8371,
15,Albany,US,75.09,overcast clouds,42.6001,-73.9662,
16,Sao Filipe,CV,79.34,overcast clouds,14.8961,-24.4956,
20,Namatanai,PG,81.23,scattered clouds,-3.6667,152.4333,
25,Mecca,SA,85.91,clear sky,21.4267,39.8261,
27,Ponta Do Sol,PT,75.70,broken clouds,32.6667,-17.1000,
30,Ambovombe,MG,77.52,clear sky,-25.1667,46.0833,
31,Ugoofaaru,MV,81.82,overcast clouds,5.6667,73.0000,


In [30]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
        
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")  
          

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [41]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()


In [35]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [38]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}<dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}<dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [39]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure


In [40]:
fig

Figure(layout=FigureLayout(height='420px'))